In [8]:
import pandas as pd

* NaN 用 0 取代
* 去除最後一筆資料 (結算最後比分)
* 加上隊伍資訊
* Action == OP 的 column 去除，reset index

In [9]:
def dataset_preprocess(df, team):
    df = df.fillna(0)
    df = df.drop(len(df)-1)
    df.insert(0, column='Team', value=[team for i in range(len(df))])
    df = df.drop(df[df['Action'] == 'OP'].index)
    df = df.reset_index(drop=True)
    return df

In [40]:
def get_data(link, team, set_num):
    df = pd.DataFrame()
    for i in range(1, set_num+1):
        new_df = pd.read_excel(link, sheet_name = team + ' Set' + str(i))
        new_df = dataset_preprocess(new_df, team)
        df = pd.concat([df, new_df])

    drop_col = df.columns.tolist()
    col = ['Team', 'No.', 'Space', 'Action', 'Erros', 'Attempts', 'Score']
    for c in col:
        drop_col.remove(c)
    df = df.drop(columns=drop_col)
    df = df.drop(df.query('Space not in [1, 2, 3, 4, 5, 6, 7, 8, 9, \'M\', \'Y\', \'L\', \'FZ\']').index)
    df = df.drop(df.query('Action not in [\'A\', \'B\', \'D\', \'FS\', \'G\', \'JS\', \'OP\', \'R\']').index)

    df = df.astype({'No.': 'int32'})
    df = df.astype({'No.': 'str'})


    df['Space'].replace({'M': 10, 'Y': 11, 'L': 12, 'FZ': 13}, inplace=True)
    
    df['Action'].replace({'A': 1, 'B': 2, 'D': 3, 'FS': 4, 'G': 5, 'JS': 6, 'OP': 7, 'R': 8}, inplace=True)

    return df

In [41]:
df_IRI = get_data('Data/2022 VNL/IRI VS POL.xlsx', 'IRI', 5)
df_POL = get_data('Data/2022 VNL/IRI VS POL.xlsx', 'POL', 5)

print(len(df_IRI), len(df_POL))

542 520


In [42]:
df_IRI.head()

,Team,No.,Space,Action,Erros,Attempts,Score
0,IRI,2,11,6,0.0,0.0,0.0
1,IRI,22,1,2,0.0,0.0,1.0
2,IRI,2,11,6,0.0,0.0,0.0
3,IRI,2,8,3,1.0,0.0,0.0
4,IRI,8,8,8,0.0,0.0,0.0


In [43]:
train_df_IRI = df_IRI.iloc[:450,:].copy()
test_df_IRI = df_IRI.iloc[450:,:].copy()

train_df_POL = df_POL.iloc[:450,:].copy()
test_df_POL = df_POL.iloc[450:,:].copy()

train_df = pd.concat([train_df_IRI, train_df_POL])
test_df = pd.concat([test_df_IRI, test_df_POL])

In [44]:
train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)